In [6]:
%%capture
!pip install scikit-learn xgboost lightgbm catboost

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings 
warnings.filterwarnings('ignore')

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.layers import LeakyReLU,PReLU,ELU
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

2024-02-17 09:52:26.472093: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-17 09:52:26.472258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-17 09:52:26.648073: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
#Loading the dataset again to revert previously made changed on BMI etc.
df_train = pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')
original = pd.read_csv('/kaggle/input/obesity-or-cvd-risk-classifyregressorcluster/ObesityDataSet.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')

In [13]:
def get_variable_types(dataframe):
    continuous_vars = []
    categorical_vars = []

    for column in dataframe.columns:
        if dataframe[column].dtype == 'object':
            categorical_vars.append(column)
        else:
            continuous_vars.append(column)

    return continuous_vars, categorical_vars

continuous_vars, categorical_vars = get_variable_types(df_train)
categorical_vars.remove('NObeyesdad')

In [15]:
train = pd.concat([df_train, original]).drop(['id'], axis=1).drop_duplicates()
test = df_test.drop(['id'], axis=1)

In [16]:
train = pd.get_dummies(train, columns=categorical_vars, drop_first=True)
test = pd.get_dummies(test, columns=categorical_vars, drop_first=True)

In [18]:
X = train.drop(['NObeyesdad'], axis=1)
y = train['NObeyesdad']

In [19]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [57]:
# Feature Scaling
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [20]:

# Define the objective function for Optuna optimization
import optuna
from optuna.samplers import TPESampler

def objective(trial, X_train, y_train, X_test, y_test):
     # Define parameters to be optimized for the LGBMClassifier
     param = {
         "objective": "multiclass",
         "metric": "multi_logloss",
         "verbosity": -1,
         "boosting_type": "gbdt",
         "random_state": 42,
         "num_class": 7,
         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.05),
         "n_estimators": trial.suggest_int("n_estimators", 400, 600),
         "lambda_l1": trial.suggest_float("lambda_l1", 0.005, 0.015),
         "lambda_l2": trial.suggest_float("lambda_l2", 0.02, 0.06),
         "max_depth": trial.suggest_int("max_depth", 6, 14),
         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.9),
         "subsample": trial.suggest_float("subsample", 0.8, 1.0),
         "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
     }

 # Create an instance of LGBMClassifier with the suggested parameters
     lgbm_classifier = LGBMClassifier(**param)
    
# Fit the classifier on the training data
     lgbm_classifier.fit(X_train, y_train)

# Evaluate the classifier on the test data
     score = lgbm_classifier.score(X_test, y_test)

     return score

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust the test_size as needed

#Set up the sampler for Optuna optimization
sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization

# Create a study object for Optuna optimization
study = optuna.create_study(direction="maximize", sampler=sampler)

# Run the optimization process
study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=3)

# Get the best parameters after optimization
best_params = study.best_params

print('='*50)
print(best_params)


[I 2024-02-17 10:11:10,472] A new study created in memory with name: no-name-589811d7-0dc2-4cb1-9b1a-80fe80146437
[I 2024-02-17 10:11:32,677] Trial 0 finished with value: 0.9124534909170496 and parameters: {'learning_rate': 0.0249816047538945, 'n_estimators': 591, 'lambda_l1': 0.012319939418114049, 'lambda_l2': 0.04394633936788146, 'max_depth': 7, 'colsample_bytree': 0.3935967122017216, 'subsample': 0.8116167224336399, 'min_child_samples': 45}. Best is trial 0 with value: 0.9124534909170496.
[I 2024-02-17 10:11:47,240] Trial 1 finished with value: 0.915517618734953 and parameters: {'learning_rate': 0.034044600469728355, 'n_estimators': 542, 'lambda_l1': 0.005205844942958024, 'lambda_l2': 0.05879639408647977, 'max_depth': 13, 'colsample_bytree': 0.4274034664069657, 'subsample': 0.8363649934414201, 'min_child_samples': 17}. Best is trial 1 with value: 0.915517618734953.
[I 2024-02-17 10:12:00,642] Trial 2 finished with value: 0.9152987524622456 and parameters: {'learning_rate': 0.0221696

{'learning_rate': 0.034044600469728355, 'n_estimators': 542, 'lambda_l1': 0.005205844942958024, 'lambda_l2': 0.05879639408647977, 'max_depth': 13, 'colsample_bytree': 0.4274034664069657, 'subsample': 0.8363649934414201, 'min_child_samples': 17}


In [59]:
# Best parameters obtained from Optuna optimization from notebook in comments
# https://www.kaggle.com/code/moazeldsokyx/pgs4e2-highest-score-lgbm-hyperparameter-tuning/notebook

best_params = {
    "objective": "multiclass",          # Objective function for the model
    "metric": "multi_logloss",          # Evaluation metric
    "verbosity": -1,                    # Verbosity level (-1 for silent)
    "boosting_type": "gbdt",            # Gradient boosting type
    "random_state": 42,       # Random state for reproducibility
    "num_class": 7,                     # Number of classes in the dataset
    'learning_rate': 0.030962211546832760,  # Learning rate for gradient boosting
    'n_estimators': 500,                # Number of boosting iterations
    'lambda_l1': 0.009667446568254372,  # L1 regularization term
    'lambda_l2': 0.04018641437301800,   # L2 regularization term
    'max_depth': 10,                    # Maximum depth of the trees
    'colsample_bytree': 0.40977129346872643,  # Fraction of features to consider for each tree
    'subsample': 0.9535797422450176,    # Fraction of samples to consider for each boosting iteration
    'min_child_samples': 26             # Minimum number of data needed in a leaf
}

In [21]:
lgbm_classifier = LGBMClassifier(**best_params)
lgbm_classifier.fit(X_train, y_train)
y_pred = lgbm_classifier.predict(X_test)

[LightGBM] [Warning] lambda_l2 is set=0.05879639408647977, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.05879639408647977
[LightGBM] [Warning] lambda_l1 is set=0.005205844942958024, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.005205844942958024
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] lambda_l2 is set=0.05879639408647977, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.05879639408647977
[LightGBM] [Warning] lambda_l1 is set=0.005205844942958024, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.005205844942958024
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003298 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you

In [22]:
accuracy_score(y_test, y_pred) 

0.9139855548260013

In [62]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, y_pred))

                     precision    recall  f1-score   support

Insufficient_Weight       0.93      0.95      0.94       574
      Normal_Weight       0.90      0.90      0.90       677
     Obesity_Type_I       0.92      0.88      0.90       682
    Obesity_Type_II       0.96      0.97      0.97       697
   Obesity_Type_III       1.00      1.00      1.00       878
 Overweight_Level_I       0.81      0.83      0.82       525
Overweight_Level_II       0.83      0.84      0.84       536

           accuracy                           0.92      4569
          macro avg       0.91      0.91      0.91      4569
       weighted avg       0.92      0.92      0.92      4569



<div style="border-radius:10px; padding: 12px; background-color: #ffffff; font-size:140%; text-align:left">
⍟ Weight, Height, Age and FAF appear to be the most important features.<br>
⍟ CH20, Time using technology devices (TUE), and NCP are the other key important features.



    
    


In [65]:
# Evaluate the best model on the test set
predictions = lgbm_classifier.predict(test)

In [66]:
submission = pd.read_csv("/kaggle/input/playground-series-s4e2/sample_submission.csv")
submission["NObeyesdad"] = predictions
submission.to_csv("submission3.csv", index=False)
submission.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III


<a id="toc"></a>

<a href="#toc" style="background-color: #E1B12D; color: #ffffff; padding: 7px 10px; text-decoration: none; border-radius: 50px;">Back to top</a><a id="toc"></a>

---